Import the libraries

In [ ]:
from dotenv import load_dotenv
from langchain import hub
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
import os
import datetime
import urllib

load_dotenv()

True

Define tools

In [2]:
from test.test_typing import Annotated


@tool("get_current_datetime")
def get_current_datetime() -> str :
    """Get current date and time"""
    return datetime.datetime.now().strftime('%m/%d/%Y, %H:%M:%S')

@tool("convert_to_japanse_datetime_custom")
def convert_to_japanse_datetime(date_str: Annotated[str, 'date string in ISO format']) -> str :
    """Convert date string to custom japanese date time"""
    dt = datetime.datetime.fromisoformat(date_str)
    return f"{dt.year}年{dt.month}月{dt.day}日　{dt.hour}時{dt.minute}分{dt.second}秒"

# dir: Annotated[str, 'the directory of the file location, pass in /tmp for default']
@tool("download_image_from_url")
def download_image_from_url(url: Annotated[str, 'full URL of the web image']) -> dict:
    "Download an image from web and save to local device"
    directory = f'{os.getcwd()}/tmp'
    file_name = f"{directory}/{datetime.datetime.timestamp(datetime.datetime.now())}-agent.png"
    cleaned_url = url.replace("'", '')
    try:
        res= urllib.request.urlretrieve(cleaned_url,file_name)    
        print(res)
        os.startfile(file_name)
        return  {
            "success": True,
            "file_name": file_name
        }
    except (Exception) as e:
         return {
                "success": False,
                "error": f"{e}",
                "url": url
            }
         
#TODO: sendemail tool
    
    

Initialized OpenAI LLM

In [52]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Initialize a ChatOpenAI model
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv('OPENAI_KEY'),
)

tools = [
    get_current_datetime,
    convert_to_japanse_datetime,
    download_image_from_url
]

# prompt = hub.pull("hwchase17/react",api_key=os.getenv('LANGSMITH_KEY'),)
memory = MemorySaver()
agent = create_react_agent(
    llm,
    tools=tools,
    # prompt=prompt,
    checkpointer=memory,
)


Testing

In [ ]:
import uuid
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": uuid.uuid4()}}
prompt1 = "Download this image for me: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS4ZNGq9_b4Z0Gw-_HXeJLlI0wOPUJnT020Xg&s"
response = agent.invoke({"messages": [HumanMessage(prompt1)]},config=config)
print("question: ", prompt1)
print("response: ", response['messages'][-1].content)
# for event in agent_executor.stream({"input": [HumanMessage(content=prompt1)]}, config):
#     print("question: ", prompt1)
#     print("response: ", event['output'])

('c:\\Users\\USER\\my-projects\\movie_react_agent/tmp/1740578831.991179-agent.png', <http.client.HTTPMessage object at 0x0000023B9F331510>)
question:  Download this image for me: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS4ZNGq9_b4Z0Gw-_HXeJLlI0wOPUJnT020Xg&s
response:  {'messages': [HumanMessage(content='Download this image for me: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS4ZNGq9_b4Z0Gw-_HXeJLlI0wOPUJnT020Xg&s', additional_kwargs={}, response_metadata={}, id='38d6270a-d7ec-433d-bce2-ee3e4686da7a'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mD9yTZI4r0xlv8n1PuSG4zdw', 'function': {'arguments': '{"url":"https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS4ZNGq9_b4Z0Gw-_HXeJLlI0wOPUJnT020Xg&s"}', 'name': 'download_image_from_url'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 159, 'total_tokens': 219, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 

In [ ]:
prompt2 = "can you download the file again?"
response = agent.invoke({"messages": [HumanMessage(prompt2)]},config=config)
print("question: ", prompt2)
print("response: ", response['messages'][-1].content)

('c:\\Users\\USER\\my-projects\\movie_react_agent/tmp/1740578889.218966-agent.png', <http.client.HTTPMessage object at 0x0000023B95A9B820>)
question:  can you download the file again?
response:  {'messages': [HumanMessage(content='Download this image for me: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS4ZNGq9_b4Z0Gw-_HXeJLlI0wOPUJnT020Xg&s', additional_kwargs={}, response_metadata={}, id='38d6270a-d7ec-433d-bce2-ee3e4686da7a'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mD9yTZI4r0xlv8n1PuSG4zdw', 'function': {'arguments': '{"url":"https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS4ZNGq9_b4Z0Gw-_HXeJLlI0wOPUJnT020Xg&s"}', 'name': 'download_image_from_url'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 159, 'total_tokens': 219, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_toke